In [2]:
import numpy as np
import pandas as pd

with open('checkins.dat', 'r') as file1:
    with open('checkins-new.dat', 'w') as file2:
        for index, line in enumerate(file1):
            newLine = line.replace(' ','').replace('|', ',')
            if (index != 1):
                file2.write(newLine)
                
data = pd.read_csv('checkins-new.dat').dropna()

C:\ProgramData\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-2117:43:47
2,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-2117:43:43
6,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-2117:39:22
8,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-2117:35:46
9,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-2117:38:18


In [6]:
data_1 = data[0:100000]

In [16]:
data_1.shape
data_cl = data_1.loc[:,["latitude", "longitude" ]]
data_cl.shape

(100000, 2)

In [9]:
from sklearn.cluster import MeanShift

In [17]:
clustering = MeanShift(bandwidth=0.1).fit(data_cl)

In [28]:
labl = clustering.labels_
cent = clustering.cluster_centers_


In [140]:
labl_u, labl_c = np.unique(labl, return_counts = True)
a = pd.DataFrame([labl_u, labl_c])
b = a.T
c = b[b>15]
cent_more = np.empty((len(c), 2))
for index, lbl in enumerate(c):
    cent_more[index] = cent[lbl]

In [72]:
office = pd.read_csv('office.txt')
office

,Latitude,Longitude
0,33.751277,-118.188740
1,25.867736,-80.324116
2,51.503016,-0.075479
3,52.378894,4.885084
4,39.366487,117.036146
5,-33.868457,151.205134


In [73]:
def distance(x,y):
    return np.sqrt(np.sum((x-y)**2))

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [ -37.8229826 ,  145.1811902 ],
       [ -41.2924945 ,  174.7732353 ],
       [ -45.0311622 ,  168.6626435 ]])

In [88]:
lol = 1000000
bb = 1
for i in range(len(cent_more)):
    for j in range(len(office)):
        a = distance(cent_more[i], office.loc[j])
        if a < lol:
            lol = a
            bb = cent_more[i]
            q = i
    j += 1
i += 1
print bb, lol, q

[  33.44943805 -112.00213969] 6.193959165601315 1


In [141]:
distances_from_nearest_office = np.empty(cent_more.shape[0])
for i, item in enumerate(cent_more):
    min_dist = distance(item, office.loc[0])
    for j in range(len(office)):
        dist = distance(item, office.loc[j])
        if dist < min_dist:
            min_dist = dist
    distances_from_nearest_office[i] = min_dist

array([   1,    0, 2148, ..., 1083, 1073, 3230], dtype=int64)